In [ ]:
# Times Square parameters
date = '2024-01-08'

# OBS issue summary

In [ ]:
import os
import json
from urllib.parse import urljoin, urlencode
from typing import Any
from pprint import pprint
from datetime import datetime
from collections import defaultdict

import httpx
import pandas as pd
from lsst.rsp import get_access_token

In [ ]:
class JiraClient:
    """Jira API client for Notebook Aspect users."""

    def __init__(self) -> None:
        env_url = os.getenv("EXTERNAL_INSTANCE_URL", None)
        if env_url is None:
            raise RuntimeError("EXTERNAL_INSTANCE_URL environment variable is not set")

        self.jira_url = urljoin(env_url, "/jira-data-proxy/")
        self._http_client = httpx.AsyncClient()

    async def aclose(self) -> None:
        await self._http_client.close()

    @property
    def _token(self) -> str:
        """An RSP token."""
        return get_access_token()

    async def search(
        self,
        jql: str,
        start_at: int | None = None,
        max_results: int | None = None,
        fields: list[str] | None = None,
        exclude_fields: list[str] | None = None,
        expand: list[str] | None = None
    ) -> dict[str, Any]:
        qs_parts = [("jql", jql)]
        if start_at:
            qs_parts.append(("startAt", str(start_at)))
        if max_results:
            qs_parts.append(("maxResults", str(max_results)))
        if fields or exclude_fields:
            all_fields = []
            if fields:
                all_fields.extend(fields)
            if exclude_fields:
                all_fields.extend([f"-{f}" for f in exclude_fields])
            qs_parts.append(("fields", ",".join(all_fields)))
        if expand:
            qs_parts.append(("exclude", ",".join(expand)))
        qs = urlencode(qs_parts)
        url = urljoin(self.jira_url, f"rest/api/2/search?{qs}")
        r = await self._http_client.get(url, headers={"Authorization": f"Bearer {self._token}"})
        r.raise_for_status()
        return r.json()


client = JiraClient()

In [ ]:
obs_data = await client.search(
    'project = "OBS" AND created >= 2024-01-15 AND created <= 2024-01-31',
    fields=["summary", "created", "status", "reporter", "components"]
)
# pprint(obs_data)

In [ ]:
reduced_obs_data = defaultdict(list)
for obs in obs_data["issues"]:
    reduced_obs_data["key"].append(obs["key"])
    fields = obs["fields"]
    reduced_obs_data["summary"].append(fields["summary"])
    reduced_obs_data["created"].append(datetime.strptime(fields["created"], "%Y-%m-%dT%H:%M:%S.%f%z"))
    reduced_obs_data["reporter"].append(fields["reporter"]["displayName"])
    reduced_obs_data["status"].append(fields["status"]["name"])
issues_df = pd.DataFrame(reduced_obs_data)

In [ ]:
issues_df.sort_values(by=["created"])